In [2]:
import os
import json
import pymongo
#import musicbrainzngs
import arrow
import re
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict, OrderedDict
import csv

from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
###BEFORE RUNNING THIS ONE YOU SHOULD CLEAN YOUR MONGO DB OF artist strings starting with &, with + and such, or remove the JSon files first"
#example :db.getCollection('selectedArtists').find({"_id": /&.*/})
#this needs to be done by hand

##Variables
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
# setup mongo
client = MongoClient()
db = client["bandstour"]
venues =  db["venues"]
artists =  db["selectedArtists"]
FUZZ_RATIO = 90 
NB_remplacements = 0
##input function
##https://gist.github.com/hrouault/1358474
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.
    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).
    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
"(or 'y' or 'n').\n")
#Clean a string with regex
def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    s = re.sub('\s+', ' ', s)
    s = s.replace(' ', '-')

    s = re.sub('\W+', ' ', s)

    s = s.strip()

    s = s.lstrip()
    s = re.sub('\W+', '_', s)
    return s

#WE WANT TO LOOP OVER ARTISTS IN ORDER TO DEDUPLICATE THE RECORDS AND ASSIGN TOURS TO ONE ENTITY ONLY.
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose, at the end
replacement_list =[]
resultFile = open("outputartistdedup.csv",'wb')
wr = csv.writer(resultFile, dialect='excel')
print "  _____________________________"
print " |_couples_d'ARTISTES_de_dups__|"
print " |_NOM_|_SLUG_|nb_dates_|_mbid_|"

for artistA in artists.find({},no_cursor_timeout=True) :
            #print "AAAA  =>", artistA["_id"]
            #print "AAAA slug  =>",slugify(artistA["_id"]).lower()
            
            for artistB in artists.find({},no_cursor_timeout=True) :
                if artistA["_id"]==artistB["_id"]:
                    pass
                elif (fuzz.ratio(\
                                        slugify(artistA["_id"]).lower()\
                                        , slugify(artistB["_id"]).lower()\
                                       ) > FUZZ_RATIO and len(artistA["gigs"]) >= len(artistB["gigs"])) :
                    print "A|", artistA["_id"],"|",slugify(artistA["_id"]).lower(),"|",len(artistA["gigs"]),"|",artistA["mbid"],"|"
                    print "B|", artistB["_id"],"|",slugify(artistB["_id"]).lower(),"|",len(artistB["gigs"]),"|",artistB["mbid"],"|"
                    print "_______________________________"
                    for gig in artistB["gigs"]:
                        artistA["gigs"].append(gig)
                        
                    Aprime = artistA["gigs"]
                    #print Aprime
                    Aprime2 = sorted(Aprime,key=lambda t: t["datetime"].strftime(DATETIME_FORMAT))
                    #print "Aprime2",Aprime2
                    #BYPASS THE FOLLOWING LINE  WHEN THE SCRIPT IS MATURE
                    #Answer_art= query_yes_no("MERGE "+artistB["_id"]+ " WITH "+ artistA["_id"]+" ?" , default="no")
                    #print Answer_art
                    Answer_art = True
                    if Answer_art == True :
                        artistA["gigs"]=Aprime2
                        #print artistA["gigs"]
                        replacement_list.append([artistA["_id"],artistB["_id"]])
                        
print "DONE!"
for  replacement in replacement_list:
    wr.writerow(replacement)
resultFile.close()

  _____________________________
 |_couples_d'ARTISTES_de_dups__|
 |_NOM_|_SLUG_|nb_dates_|_mbid_|
A|    Broken      | broken | 1 | None |
B| BROKEN2 | broken2 | 1 | None |
_______________________________
A|    Broken      | broken | 2 | None |
B| Broken | broken | 1 | 1189c81d-d276-45dd-9be0-1dc37e67e511 |
_______________________________
A|   Bobaflex | bobaflex | 1 | None |
B| Bobaflex. | bobaflex | 1 | None |
_______________________________
A|  **Prophet** | prophet | 1 | None |
B| Prophet | prophet | 1 | f3910f01-f99d-4145-915b-6030c89a354b |
_______________________________
A|  A Day to Remember | a_day_to_remember | 2 | None |
B| A Day To Remeber | a_day_to_remeber | 1 | None |
_______________________________
A|  A Skylit Drive | a_skylit_drive | 1 | None |
B| A Skykit Drive | a_skykit_drive | 1 | None |
_______________________________
A|  A Skylit Drive | a_skylit_drive | 2 | None |
B| Askylitdrive | askylitdrive | 1 | None |
_______________________________
A|  Abigail, Speedwolf 

A|  JIM WEIDER's PrOJECT PERCoLATOR | jim_weider_s_project_percolator | 2 | None |
B| JIM WEIDER'S PrOJECT PERCoLAT0R  | jim_weider_s_project_percolat0r | 1 | None |
_______________________________
A|  JIM WEIDER's PrOJECT PERCoLATOR | jim_weider_s_project_percolator | 3 | None |
B| JIM WEIDERS  PRoJECT PERCoLAToR | jim_weiders_project_percolator | 1 | None |
_______________________________
A|  JIM WEIDER's PrOJECT PERCoLATOR | jim_weider_s_project_percolator | 4 | None |
B| JIM WEIDERS PROJECT PERCOLATOR | jim_weiders_project_percolator | 2 | None |
_______________________________
A|  JIM WEIDER's PrOJECT PERCoLATOR | jim_weider_s_project_percolator | 6 | None |
B| Jim Weider and PRoJECT PERCoLAToR | jim_weider_and_project_percolator | 1 | None |
_______________________________
A|  JONATHAN SCALES FOURCHESTRA  | jonathan_scales_fourchestra | 1 | None |
B| John Scales Fourchestra | john_scales_fourchestra | 1 | None |
_______________________________
A|  Jacob Luttrell | jacob_luttrell 